<a href="https://colab.research.google.com/github/mahbodnr/Mask-Detection/blob/main/Project_Brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO:

- <s>neuron id </s>
- define a global timestep

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import count
import networkx as nx

In [2]:
dt = 0.001 #s
T = 0.1 #s
total_timepoints = int(T/ dt)

LIF:<br>
$u = u_r + \int 1/C * exp(t-t'/\tau) * I(t')dt'$

In [ ]:
class NeuronType(object):
    pass

class LIF(NeuronType):
    def __init__(self,  Rm = 1,Cm = 0.1):
        self.Rm = Rm #ohm
        self.Cm = Cm #uF
        self.tau_m = Rm*Cm

    def __call__(self, current, timestep, last_spike_timepoint,):
        potential = 0
        for timepoint in range(last_spike_timepoint, timestep + 1):
            potential += 1/self.Cm * (np.exp(-(timestep - timepoint)/self.tau_m)) * current[timepoint]
        return potential

In [113]:
class Neuron(object):
    _ids = count(0)
    def __init__(self, total_timepoints,  model = LIF(),
                 neurotransmitter = 'inhibitory', tau_ref = 4, u_rest = -68,
                 u_tresh = +30, save_potential_history = False,
                 base_current = 1):
        """
        Define a new Neuron object
        Args:

        """
        self.id = next(self._ids)
        self.model = model
        self.neurotransmitter = neurotransmitter
        self.tau_ref = tau_ref #ms # refractory period
        self.u_rest = u_rest #mv
        self.u_tresh = u_tresh #mv
        self.base_current = base_current #mv
        self.u = self.u_rest
        self.save_potential = save_potential_history
        self.open = True
        self.current_history = np.zeros(total_timepoints)
        self.spike_train =  np.zeros(total_timepoints, dtype = np.bool)
        self.last_spike_timepoint = 0
        if self.save_potential:
            self.potential = np.zeros(total_timepoints)
        self.current = 0
        self.timestep = 0
        
    def step(self):
        assert self.timestep < self.total_timepoints, "Simulation interval has been finished!"
        # Check refactory interval
        if not open:
            if self.refactory_time < self.tau_ref:
                self.refactory_time += 1
            else:
                self.open = True
        else:
            # Add current current to current history :)
            self.current_history[self.timestep] = self.current
            # Update
            self.u = self.u_rest
            self.u += self.model(self.current_history, self.timestep, self.last_spike_timepoint)
            # Save potential history
            if self.save_potential:
                self.potential[self.timestep] = self.u
            # Spike
            if self.u > self.u_tresh:
                self.last_spike_timepoint = self.timestep + self.tau_ref
                self.spike_train[self.timestep] = True
                self.open = False
                self.refactory_time = 0 
        # Go to next time step
        self.current = 0

    @property
    def spike(self):
        return self.spike_train[self.timestep - 1]

In [114]:
class NeuronGroup(object):
    def __init__(self, population, total_timepoints,
                 connection_chance, excitatory_rate):
        self.population = population
        self.neurons = {
            Neuron(total_timepoints, 
                   neurotransmitter = 'inhibitory' if random.random() > excitatory_rate else 'excitatory')
            for _ in range(self.population)}
                

        self.network = nx.DiGraph()
        self.network.add_nodes_from(self.neurons)
        for PreSN in self.network.nodes:
            for PostSN in self.network.nodes:
                if PreSN != PostSN:
                    if random.random() < connection_chance:
                        self.network.add_edge(PreSN, PostSN, weight = np.random.normal(scale= 1))

        self.timestep = 0

    def step(self):
        for neuron in self.neurons:
            if neuron.current:
                neuron.step()
                neuron.timestep = self.timestep

        for neuron in self.neurons:
            if neuron.spike:
                for _, postSN, weight in self.network.out_edges(neuron, data = 'weight'):
                    if postSN.open:
                        postSN.current += self.base_current * weight


In [115]:
class stimulus(object):
    _ids = count(0)
    def __init__(self, output, dt):
        """
        Parameters
        ----------
        output: function
        A function that determines the output of the stimulus
        in a specific timestep.
        examples:
        output = lambda t: np.sin(2 * t)
        output = lambda t: 2

        Returns
        -------
        Returns the value of output if no connection is set.
        otherwise, sends he value to the connection (returns None).
        """
        self.output = output
        self.connection = None
        self.id = next(self._ids)
        self.timestep = 0
        self.dt = dt

    def connect(self, connection):
        self.connection = connection

    def step(self):
        if self.connection == None:
            print(f"WARNING: Stimulus (id: {self.id}) has not connected to any object!")
            return self.output(self.timestep * self.dt)
        self.connection.current += self.output(self.timestep * self.dt)


In [116]:
class simulation(object):
    def __init__(self, total_time, dt):
        self.total_timepoints = int(T/ dt)
        self.timepoint = 0
        self.objects = set()
        self.dt = dt

    def NeuronGroup(self, population, connection_chance=1/10,
                    excitatory_rate=2 /10):
        G = NeuronGroup(population, self.total_timepoints,
                        connection_chance, excitatory_rate)
        self.objects.add(G)
        return G

    def stimulus(self, output):
        stim = stimulus(output, self.dt)
        self.objects.add(stim)
        return stim

    def step(self):
        for obj in self.objects:
            obj.step()
            obj.timestep +=1

    @property
    def time(self):
        return self.timepoint * self.dt

#Run

In [122]:
sim = simulation(total_time= 1, dt = 0.001)
group = sim.NeuronGroup(10)
stim = sim.stimulus(lambda t : np.sin(t))
stim.connect(next(iter(group.neurons)))
sim.step()